In [1]:
import re
import sys
import csv
import pandas as pd
from collections import Counter

PATHS:

In [2]:
#irony speech:
IRONY_RAW_PATH = '../datasets/irony/raw/'
IRONY_INTERIM_PATH = '../datasets/irony/interim/'

#emoji:
EMOJI_RAW_PATH = '../datasets/emoji/raw/'
EMOJI_INTERIM_PATH = '../datasets/emoji/interim/'

FILES:

In [3]:
TEST = 'test_text.txt'
TRAIN = 'train_text.txt'
VAL = 'val_text.txt'

TEST_LABELS = 'test_labels.txt'
TRAIN_LABELS = 'train_labels.txt'
VAL_LABELS = 'val_labels.txt'


TEST_INTERIM = 'test_seperated.csv'
TRAIN_INTERIM = 'train_seperated.csv'
VAL_INTERIM = 'val_seperated.csv'

FUNCTIONS:

In [4]:
def tokenise(line):
    # Initialise lists
    tokens = []
    unmatchable = []

    # Compile patterns for speedup
    token_pat = re.compile(r'\w+|#+|\'|@|\.\.+|!+|\?+')
    skippable_pat = re.compile(r',|\|http://t.co/+')  # typically spaces

    # As long as there's any material left...
    while line:
        # Try finding a skippable token delimiter first.
        skippable_match = re.search(skippable_pat, line)
        if skippable_match and skippable_match.start() == 0:
            # If there is one at the beginning of the line, just skip it.
            line = line[skippable_match.end():]
        else:
            # Else try finding a real token.
            token_match = re.search(token_pat, line)
            #print(token_match)
            if token_match and token_match.start() == 0:
                #print(line[token_match.start():token_match.end()])
                if line[token_match.start():token_match.end()] == '#': #keep hash tags together and seperate
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        if ' ' in line[token_match2.start():token_match2.end()]:
                            line = line[token_match.end():]
                        else:
                            tokens.append(line[:token_match2.end()+1])
                            line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                elif line[token_match.start():token_match.end()] == '@': #keep @ tags together and seperate
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        if ' ' in line[token_match2.start():token_match2.end()]:
                            line = line[token_match.end():]
                        
                        else: 
                            tokens.append(line[:token_match2.end()+1])
                            line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                elif line[token_match.start():token_match.end()] == "'": #handle contractions as a single word
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        if ' ' in line[token_match2.start():token_match2.end()]:
                            line = line[token_match.end():]
                        
                        else: 
                            tokens.append(line[:token_match2.end()+1])
                            line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                # If there is one at the beginning of the line, tokenise it.
                else:
                    tokens.append(line[:token_match.end()])
                    line = line[token_match.end():]
            else:
                # Else there is unmatchable material here.
                # It ends where a skippable or token match starts, or at the end of the line.
                unmatchable_end = len(line)
                if skippable_match:
                    unmatchable_end = skippable_match.start()
                if token_match:
                    unmatchable_end = min(unmatchable_end, token_match.start())
                # Add it to unmatchable and discard from line.
                unmatchable.append(line[:unmatchable_end])
                line = line[unmatchable_end:]

    final_tokens = []

    while len(tokens) > 0:
        temp1 = tokens.pop(0)
        try:
            temp2 = tokens.pop(0)
            if temp2[0] == "'":
                temp1 += temp2
                final_tokens.insert(0, temp1)
                #print('a', temp1)
            else:
                final_tokens.insert(0, temp1)
                tokens.insert(0, temp2)
                #print('b', temp1, temp2)
        except:
            final_tokens.insert(0, temp1)
            #print('d', temp1)
        
    final_tokens = final_tokens[::-1]

    #print(final_tokens)
    #print(unmatchable)
    return final_tokens

In [5]:
f = open(IRONY_RAW_PATH + TRAIN, "r", encoding="utf-8")
final_irony = []
for line in f:
    final_irony.append(tokenise(line))
f.close()

with open(IRONY_INTERIM_PATH+TRAIN_INTERIM, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(final_irony)

In [6]:
final_irony[3]

['@user',
 '@user',
 'So',
 'is',
 'he',
 'banded',
 'from',
 'wearing',
 'the',
 'clothes',
 '?',
 '#Karma']

In [7]:
temp1 = pd.read_csv(IRONY_INTERIM_PATH+TRAIN_INTERIM, delimiter="\n", names=['tweet'])
temp2 = pd.read_csv(IRONY_RAW_PATH+TRAIN_LABELS)
irony_df = temp1.merge(temp2, left_index=True, right_index=True)

In [8]:
print(irony_df.shape)

count_irony = Counter()

for line in final_irony:
    count_irony.update(line)

count_irony.most_common()

(2861, 2)


[('@user', 1731),
 ('the', 964),
 ('to', 934),
 ('a', 757),
 ('I', 724),
 ('is', 483),
 ('and', 470),
 ('!', 464),
 ('in', 440),
 ('of', 426),
 ('you', 392),
 ('for', 387),
 ('my', 373),
 ('?', 343),
 ('...', 338),
 ('on', 285),
 ('that', 275),
 ('it', 270),
 ('be', 198),
 ('me', 188),
 ('this', 184),
 ('with', 182),
 ('at', 181),
 ('have', 180),
 ('so', 177),
 ('are', 164),
 ('not', 158),
 ('up', 155),
 ('just', 143),
 ("I'm", 141),
 ('all', 138),
 ('was', 134),
 ('but', 129),
 ('your', 128),
 ('like', 126),
 ('love', 124),
 ('get', 121),
 ('when', 117),
 ('..', 112),
 ('from', 103),
 ('about', 103),
 ('day', 102),
 ('do', 100),
 ('can', 99),
 ('we', 95),
 ('The', 95),
 ('how', 93),
 ('an', 92),
 ('by', 89),
 ('out', 88),
 ('as', 86),
 ('now', 85),
 ('great', 81),
 ('will', 80),
 ('what', 80),
 ('one', 79),
 ('they', 77),
 ('time', 76),
 ('2', 75),
 ("don't", 75),
 ('people', 74),
 ('he', 71),
 ("it's", 71),
 ('more', 71),
 ('know', 68),
 ('has', 68),
 ('!!', 67),
 ('going', 65),
 ('o

In [9]:
counter_irony_1 = 0
counter_irony_ = 0
for k,v in count_irony.items():
    if v == 1:
        counter_irony_1 += 1
    elif v > 1:
        counter_irony_ += 1
        
print(f'number of words that occur only once: {counter_irony_1}, ({counter_irony_1/(counter_irony_1 + counter_irony_):.0%})\nnumber of words that occur more than once: {counter_irony_}, \ntotal word count, including repeated words: {sum(count_irony.values())}')

number of words that occur only once: 7250, (70%)
number of words that occur more than once: 3050, 
total word count, including repeated words: 40371


In [10]:
count_irony.most_common()[:-100-1:-1] #least common words

[('dues', 1),
 ('shorted', 1),
 ('union', 1),
 ('#skincare', 1),
 ('heaven', 1),
 ('products', 1),
 ('thot', 1),
 ('#practicewhatyoupreach', 1),
 ('cracks', 1),
 ('managers', 1),
 ('#atheism', 1),
 ('#atheist', 1),
 ('beliefs', 1),
 ('Funk', 1),
 ('#wordsofkindness', 1),
 ('impart', 1),
 ('#countonmother', 1),
 ('beta', 1),
 ('gain', 1),
 ('Signup', 1),
 ('Born', 1),
 ('Engineered', 1),
 ('Imagined', 1),
 ('Vacations', 1),
 ('#person', 1),
 ('#boring', 1),
 ('#am', 1),
 ('fr', 1),
 ('dry', 1),
 ('Uhhh', 1),
 ('#TooMuch', 1),
 ("see'em", 1),
 ('lust', 1),
 ('lil', 1),
 ('briefs', 1),
 ('requires', 1),
 ('necklaces', 1),
 ('pearl', 1),
 ('classiest', 1),
 ('urgency', 1),
 ('Bears', 1),
 ('Criminal', 1),
 ('Thug', 1),
 ('Unbelievable', 1),
 ('#chore', 1),
 ('Anything', 1),
 ('Suppose', 1),
 ('#MensFashion', 1),
 ('sorts', 1),
 ('beard', 1),
 ('Goodbye', 1),
 ('makin', 1),
 ('heartbeat', 1),
 ('Navi', 1),
 ("friend's", 1),
 ('kidnap', 1),
 ('asylum', 1),
 ('seeking', 1),
 ('#LGBT', 1),
 ('

In [11]:
f = open(EMOJI_RAW_PATH + TRAIN, "r", encoding="utf-8")
final_emoji = []
for line in f:
    final_emoji.append(tokenise(line))
f.close()

with open(EMOJI_INTERIM_PATH+TRAIN_INTERIM, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(final_emoji)

In [12]:
final_emoji[16038]

['My',
 'Girl',
 'my',
 'Girl',
 '!!!!',
 'She',
 'loved',
 'this',
 'one',
 'the',
 'BEST',
 '!!!',
 'Did',
 'a',
 'new',
 'cut',
 'on',
 'her',
 '#BLESSED',
 '##',
 'shorthair']

In [13]:
temp1 = pd.read_csv(EMOJI_INTERIM_PATH+TRAIN_INTERIM, delimiter="\n", names=['tweet'])
temp2 = pd.read_csv(EMOJI_RAW_PATH+TRAIN_LABELS, names=['label'])
emoji_df = temp1.merge(temp2, left_index=True, right_index=True)

In [14]:
print(emoji_df.shape) #should be 45000


count_emoji = Counter()

for line in final_emoji:
    count_emoji.update(line)

count_emoji.most_common()

(45000, 2)


[('@user', 12209),
 ('the', 10346),
 ('!', 9045),
 ('to', 7645),
 ('I', 6535),
 ('a', 6100),
 ('my', 5965),
 ('in', 5688),
 ('and', 5389),
 ('with', 5049),
 ('of', 5016),
 ('you', 4774),
 ('for', 4698),
 ('California', 3991),
 ('this', 3563),
 ('at', 3454),
 ('The', 3385),
 ('is', 3381),
 ('on', 2598),
 ('...', 2240),
 ('love', 2134),
 ('amp', 2131),
 ('it', 1967),
 ('me', 1966),
 ('Los', 1916),
 ('San', 1865),
 ('Angeles', 1750),
 ('so', 1639),
 ('My', 1601),
 ('from', 1535),
 ('!!', 1534),
 ('day', 1513),
 ('was', 1463),
 ('Beach', 1451),
 ('be', 1372),
 ('by', 1328),
 ('all', 1309),
 ('your', 1309),
 ('Happy', 1286),
 ('Vegas', 1273),
 ('?', 1267),
 ('out', 1237),
 ('that', 1214),
 ('up', 1179),
 ('are', 1168),
 ('today', 1166),
 ('night', 1165),
 ('our', 1151),
 ('time', 1135),
 ("I'm", 1115),
 ('Christmas', 1103),
 ('!!!', 1061),
 ('Las', 1053),
 ('This', 987),
 ('we', 983),
 ('like', 960),
 ('have', 955),
 ('one', 935),
 ('Park', 922),
 ('CA', 921),
 ('A', 909),
 ('Hollywood', 87

In [15]:
counter_emoji_1 = 0
counter_emoji_ = 0
for k,v in count_emoji.items():
    if v == 1:
        counter_emoji_1 += 1
    elif v > 1:
        counter_emoji_ += 1
        
print(f'number of words that occur only once: {counter_emoji_1}, ({counter_emoji_1/(counter_emoji_1 + counter_emoji_):.0%})\nnumber of words that occur more than once: {counter_emoji_} \ntotal word count, including repeated words: {sum(count_emoji.values())}')

number of words that occur only once: 49420, (69%)
number of words that occur more than once: 21953 
total word count, including repeated words: 511305


In [16]:
count_emoji['# #'], count_emoji['##']

(0, 16)

In [17]:
count_emoji.most_common()[:-200-1:-1] #Least common words are often personal names or hashtags.

[('Five50', 1),
 ('#happylaborday', 1),
 ('Nunez', 1),
 ('#bffweekend', 1),
 ('Players', 1),
 ('#rolltide', 1),
 ('#Alabama', 1),
 ('chefking1921express', 1),
 ('nector', 1),
 ('thedabberchick', 1),
 ('#southbayla', 1),
 ('Corban', 1),
 ('#squints', 1),
 ('#bæ', 1),
 ('Brody', 1),
 ('Catcher', 1),
 ('hangz', 1),
 ('traivs_', 1),
 ('#mountaingirl', 1),
 ('#WhatsHotOnTheBlock', 1),
 ('#Convoy', 1),
 ('#KoreanBBQ', 1),
 ('#overcast', 1),
 ('#oneblockaway', 1),
 ('Skyshow', 1),
 ('#bluewall', 1),
 ('#bluewallpics', 1),
 ('Hoppin', 1),
 ('#teamlobsta', 1),
 ('dee_bbyy052', 1),
 ('Meadowbrook', 1),
 ('#ohsostearns', 1),
 ('#bloodlust', 1),
 ('#familyparty', 1),
 ('learner', 1),
 ("Beau's", 1),
 ('Calaveras', 1),
 ('LaSalette', 1),
 ('Fishermen', 1),
 ('diffusing', 1),
 ('Snapbacks', 1),
 ('#80sstyle', 1),
 ('#offthewall', 1),
 ('tkfotos', 1),
 ('#trusouljatv', 1),
 ('TruSouljaTV', 1),
 ('Moonwalking', 1),
 ('#notmysport', 1),
 ('#wegoeveryyear', 1),
 ('Guemes', 1),
 ('#pchobbies', 1),
 ('#pd

In [18]:
for i in range(20):
    print(i, ': ', (emoji_df[emoji_df['label'] == i]).shape[0])

0 :  9204
1 :  4901
2 :  4713
3 :  2043
4 :  2146
5 :  2132
6 :  2078
7 :  2345
8 :  1287
9 :  1391
10 :  1982
11 :  946
12 :  1246
13 :  980
14 :  1224
15 :  934
16 :  1350
17 :  1397
18 :  1510
19 :  1191


In [19]:
emoji_df.head()

,tweet,label
0,"Sunday,afternoon,walking,through,Venice,in,the...",12
1,"Time,for,some,BBQ,and,whiskey,libations,Chomp,...",19
2,"Love,love,love,all,these,people,#friends,#bff,...",0
3,"Toys,R,Us",0
4,"Man,these,are,the,funniest,kids,ever,!!,That,f...",2


In [20]:
temp1 = pd.read_csv(IRONY_INTERIM_PATH+TRAIN_INTERIM, delimiter="\n", names=['tweet'])
temp2 = pd.read_csv(IRONY_RAW_PATH+TRAIN_LABELS, names=['label'])
irony_df = temp1.merge(temp2, left_index=True, right_index=True)

In [21]:
print('Ironic:', (irony_df[irony_df['label'] == 1]).shape[0], '\n Other:', (irony_df[irony_df['label'] == 0]).shape[0])

Ironic: 1445 
 Other: 1417


In [22]:
irony_df.head()

,tweet,label
0,"seeing,ppl,walking,w,crutches,makes,me,really,...",1
1,"look,for,the,girl,with,the,broken,smile,ask,he...",0
2,"Now,I,remember,why,I,buy,books,online,@user,#s...",1
3,"@user,@user,So,is,he,banded,from,wearing,the,c...",1
4,"Just,found,out,there,are,Etch,A,Sketch,apps,#o...",1


### Interrater stuff

In [ ]:
from random import randint

In [106]:
interrater_df = pd.read_csv('../datasets/iaa-sets/irony/iaa_labels.txt', names = ['True_label'])
rater1 = pd.read_csv('../datasets/iaa-sets/irony/DeeAnnotation.txt', names = ['Dee'])
rater2 = pd.read_csv('../datasets/iaa-sets/irony/SannaAnnotation.txt', names = ['Sanna'])
rater3 = pd.read_csv('../datasets/iaa-sets/irony/GinoAnnotation.csv', names = ['Gino']).reset_index().drop('index', axis= 1)

interrater_df = interrater_df.merge(rater1, left_index=True, right_index = True)
interrater_df = interrater_df.merge(rater2, left_index=True, right_index = True)
interrater_df = interrater_df.merge(rater3, left_index=True, right_index = True)

rater_list = ['Dee', 'Sanna', 'Gino']

interrater_df.shape

(120, 4)

In [109]:
print(interrater_df['True_label'].sum())
print(interrater_df['Dee'].sum())
print(interrater_df['Sanna'].sum())
print(interrater_df['Gino'].sum())

55
44
25
51


In [108]:
interrater_df

,True_label,Dee,Sanna,Gino
0,1,1,1,1
1,1,0,0,0
2,1,0,0,0
3,0,0,0,1
4,1,1,0,0
...,...,...,...,...
115,0,0,0,0
116,0,1,1,0
117,1,0,1,0
118,1,1,0,0
